# Fitting a GLM to Estimate Residual Deaths

## Potential Pipeline

1. Consider the generalized additive model

$$y_t = \alpha + X_t\beta_t + Z\theta_t + f(t) + \varepsilon_t$$

For a give county, let 
$$
\begin{align}
    u_t &= \frac{\text{deaths}}{ \text{population total}} \text{ at time } t \\
    \alpha &= \text{constants} \\
    X_t &= \text{Lagged mobilities} \text{(Also could consider incorporating $\log u_{t-1}$)}\\
    Z &= \text{constant confounders (i.e. interventions)} \\
    \beta_t, \theta_t &= \text{coefficients to learn}\\
    f(t) &= \text{baseline death curve} \\
    \varepsilon_t &= \text{noise}
\end{align}
$$
Then we model the confounder explained deaths as

$$ y_t = \log(u_t) = \alpha + X_t\beta_t + Z\theta_t$$

The residuals $f(t) + \varepsilon_t = \log(u_t) - (\alpha + X_t\beta_t + Z\theta_t)$ are noisy baseline death curves.

2. Cluster using time-series appropriate k-means (DTW metric)

BIC or AIC to identify best set of clusters

3. Explantory regression or exploratory data analysis within each cluster

## Comments

Requires all death time series to be the same length. Thus, need to truncate and only allow counties with that many timesetps after >5 deaths observed. Is that threshold cumulative or daily?

In [315]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
from matplotlib import dates
import matplotlib.pyplot as plt
import math
import seaborn as sns
import sys; sys.path.append('../')
from src.data_loader.data_loader import load_google_mobility, load_deaths, load_interventions, load_counties, load_google_mobility_time_series
from src.utils.dates import get_today, lag_date, date2str, str2date, get_format
from src.utils.df_utils import get_date_columns
from src.pandas.align import align_lagged_dates

from scipy.stats import spearmanr

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
# Time series data
mobility, mobility_date = load_google_mobility()
deaths, deaths_date = load_deaths(join_county_codes=False)
interventions, interventions_date = load_interventions()

# Static data
counties, counties_date = load_counties()

# Processed mobility -> time series
mobility_ts, mobility_ts_date = load_google_mobility_time_series()

print(f'Mobility Last Updated {mobility_ts_date}')
print(f'Deaths Last Updated {deaths_date}')

Mobility Last Updated 04-26
Deaths Last Updated 04-26


In [52]:
# Ignore places with FIPS missing
deaths = deaths.dropna(axis=0, subset=['FIPS']).astype({'FIPS':'int32'})
deaths = deaths.merge(counties['FIPS'], on='FIPS', how='inner')

In [147]:
# Get columns that are dates
death_dates = get_date_columns(deaths, return_dtimes=False)
mobility_dates = get_date_columns(mobility_ts, return_dtimes=False)

In [ ]:
deaths = deaths[['FIPS']+death_dates]

In [68]:
def get_onset_date(row, thresh = 5):
    above = row[row > thresh]
    if len(above) == 0:
        return np.nan
    else:
        return above.idxmin()

In [72]:
deaths['onset'] = deaths[death_dates].apply(lambda row: get_onset_date(row), axis=1)

In [74]:
deaths = deaths.dropna(axis=0, subset=['onset'])

In [80]:
date_diffs = [(str2date(death_dates[-1]) - str2date(d)).days for d in deaths['onset']]

In [107]:
def normalize(row):
    FIPS = row['FIPS']
    pop = counties[counties['FIPS'] == FIPS]['POP_ESTIMATE_2018']
    return(row[death_dates] / int(pop))

In [109]:
## Normalize deaths by pop total
deaths[death_dates] = deaths.apply(lambda row: normalize(row), axis=1)

## Get deaths, lagged mobility, and covariates

In [ ]:
date_diffs = [(str2date(death_dates[-1]) - str2date(d)).days for d in deaths['onset']]

In [121]:
deaths_df = deaths[deaths['onset'].apply(lambda d: (str2date(death_dates[-1]) - str2date(d)).days >= 14)]

In [132]:
# Get days_past the onset of each county (onset being first case of deaths==5)
days_past = 0
yt = deaths_df.apply(lambda row: row[lag_date(row['onset'], days_past, backwards=False, return_date=False)], axis=1)

In [203]:
def get_lagged_mobility(date, FIPS):
    mob_dates = [lag_date(date, l, return_date=False) for l in range(13,25,1)]
    try:
        return(mobility_ts[mobility_ts['FIPS'] == FIPS][mob_dates])
    except:
        np.nan

In [326]:
## Make covariate matrices
constant_covariates = deaths_df[['FIPS', 'onset']]
covariates = deaths_df[['FIPS', 'onset']]

In [266]:
## Time dependent lagged mobility
covariates['lagged_mob'] = covariates.apply(lambda row: get_lagged_mobility(
        lag_date(row['onset'], days_past, backwards=False, return_date=False), row['FIPS']
), axis=1)

In [267]:
covariates = covariates.dropna(axis=0, subset=['lagged_mob'])
covariates['lagged_mob'] = covariates['lagged_mob'].apply(lambda x: x.to_numpy().reshape(-1))
covariates = covariates[covariates['lagged_mob'].apply(lambda x: len(x) == 12)]

In [335]:
## Constant covariates of interest
intervention_covs = [
    'stay at home',
    'restaurant dine-in',
    'public schools',
    '>50 gatherings',
    'entertainment/gym'
]

county_covs = [
    'Rural-urban_Continuum Code_2013',
    'Urban_Influence_Code_2013',
    'Density per square mile of land area - Population',
    'Total_age65plus',
    'Total Hospitals (2019)',
    'MEDHHINC_2018'
]

In [327]:
## Add intervention covariates
constant_covariates = constant_covariates.merge(interventions[['FIPS'] + intervention_covs], on='FIPS')

In [328]:
## Set intervention date to days before onset
constant_covariates[intervention_covs] = constant_covariates.apply(lambda row: pd.Series([(str2date(row['onset']) - str2date(x)).days if not pd.isna(x) else 0 for x in row[intervention_covs]]),axis=1)

In [337]:
## Add county covariates
constant_covariates = constant_covariates.merge(counties[['FIPS'] + county_covs], on='FIPS')

In [341]:
## Normalize age65plus by total population
constant_covariates['Total_age65plus'] = constant_covariates.apply(lambda row: row['Total_age65plus'] / int(counties[counties['FIPS'] == row['FIPS']]['POP_ESTIMATE_2018']),axis=1)

In [342]:
constant_covariates

,FIPS,onset,stay at home,restaurant dine-in,public schools,>50 gatherings,entertainment/gym,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2013,Density per square mile of land area - Population,Total_age65plus,Total Hospitals (2019),MEDHHINC_2018
0,1017,04-04,0,16,19,15,7,6.0,5.0,57.4,0.195210,0.694600,39917.0
1,1055,04-11,7,23,26,22,14,3.0,2.0,195.2,0.190115,2.118018,44903.0
2,1073,04-03,8,15,18,14,6,1.0,1.0,592.5,0.158573,13.623375,55013.0
3,1081,04-10,6,22,25,21,13,3.0,2.0,230.8,0.118829,3.387577,48937.0
4,1097,04-05,1,17,20,16,8,2.0,2.0,335.9,0.162030,8.549624,43598.0
5,4005,04-02,2,14,17,16,14,3.0,2.0,7.2,0.124988,1.653300,60187.0
6,4013,04-02,2,14,17,16,14,1.0,1.0,414.9,0.151737,51.048029,65234.0
7,4017,04-08,8,20,23,22,20,4.0,5.0,10.8,0.181176,1.278219,40650.0
8,4019,04-02,2,14,17,16,14,2.0,2.0,106.7,0.197537,12.025560,53395.0
9,5119,04-09,0,21,23,14,21,2.0,2.0,503.8,0.155534,11.465775,49957.0
